In [1]:
import pandas as pd

df = pd.DataFrame()

In [2]:
import os

In [3]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

420

In [4]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import re

In [5]:
for a in tqdm(range(len(list_file)), desc='Progress', ncols=77):
    with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
        resp = file.read()
        
    # = = = = = = = = = = = = = = =
    
    soup = BeautifulSoup(resp, 'lxml')
    html = etree.HTML(str(soup).encode('ascii', 'xmlcharrefreplace').decode('utf-8'))
    
    # = = = = = = = = = = = = = = =
    
    title = html.xpath('//h1[@id="title"]/span/text()')[0].strip()

    # = = = = = = = = = = = = = = =

    price = html.xpath('//span[@class="a-offscreen"]/text()')[1].strip()
    
    # = = = = = = = = = = = = = = =
    
    about = ''
    list_about = html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')
    for i in range(len(list_about)):
        about += str(i+1) + '. ' + list_about[i].strip() + '\n'

    # = = = = = = = = = = = = = = =
    
    src = html.xpath('//div[@id="imgTagWrapperId"]/img/@src')[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    list_tr = html.xpath('//table[@class="a-normal a-spacing-micro"]/tr')

    # = = = = = = = = = = = = = = =
    
    list_tr_2 = html.xpath('//table[@id="productDetails_techSpec_section_1"]/tr')
            
    # = = = = = = = = = = = = = = =
    
    list_tr_3= html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tr')

    # = = = = = = = = = = = = = = =
    
    df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                             'ASIN': '',
                             'Title': title,
                             'Price': price,
                             'About': about.strip(),
                             'Pic': '',
                             'Url': '',
                             'Src': src}])

    for tr in list_tr:
        df_temp.loc[0, tr.xpath('./td/span/text()')[0].strip().upper()] = tr.xpath('./td/span/text()')[1].strip()

    for tr in list_tr_2:
        df_temp.loc[0, tr.xpath('./th/text()')[0].strip().lower()] = tr.xpath('./td/text()')[0].strip()

    for tr in list_tr_3:
        if tr.xpath('./th/text()')[0].strip() == 'Best Sellers Rank':
            df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/descendant::*/text()')).strip()
        else:
            df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/text()')).strip()


    df = pd.concat([df, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =
    
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|████████████████████████████| 420/420 [01:28<00:00,  4.75it/s]


,序号,ASIN,Title,Price,About,Pic,Url,Src,BRAND,MATERIAL,...,exterior,WHEEL SIZE,VEHICLE SERVICE TYPE,outside diameter,voltage,COLOR,STYLE,wheel size,vehicle service type,compatible lubricant
0,1,B006WQRGI0,Detroit Axle - 4WD Front Wheel Bearing Hub for...,$59.69,1. Wheel Hub Assembly Front Replacement for 20...,,,https://m.media-amazon.com/images/I/71peMRjqyd...,Detroit Axle,Acrylonitrile Butadiene Styrene (ABS),...,,,,,,,,,,
1,2,B00HHGT34E,Detroit Axle - 4WD Front Wheel Bearing Hubs fo...,$113.58,1. Wheel Bearing Hubs Replacement for 4WD / 4x...,,,https://m.media-amazon.com/images/I/71oYneXQf4...,,,...,,,,,,,,,,
2,3,B01CURQ8IS,Detroit Axle - 4WD Front Wheel Bearing Hub for...,$58.34,1. For 4WD Models Only\n2. Wheel Bearing Hub R...,,,https://m.media-amazon.com/images/I/61OdjXciM6...,,,...,‎Smooth,,,,‎12 Volts,,,,,
3,4,B01CURQ8W4,Detroit Axle - 4WD Front Wheel Bearing Hubs fo...,$103.65,1. ONLY for 4x4 / AWD / 4WD Models - [Installa...,,,https://m.media-amazon.com/images/I/71eYrXlfgm...,,,...,‎Smooth,,,,,,,,,
4,5,B00HHGRXA0,Detroit Axle - 2 Front Wheel Bearing Hubs for ...,$86.66,1. Wheel Bearing Hubs Replacement for 2004-200...,,,https://m.media-amazon.com/images/I/71bxk2pXYV...,Detroit Axle,Acrylonitrile Butadiene Styrene (ABS),...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,416,B0BMT4J9TK,Detroit Axle - Front 2pc Wheel Bearing Hubs fo...,$105.97,1. Replacement for 2011-2017 Honda Odyssey\n2....,,,https://m.media-amazon.com/images/I/717c7K6anV...,,,...,,,,,,,,,,
416,417,B07K9S3TPS,Detroit Axle - 4 Wheel Bearing Hubs for 2006-2...,$188.79,1. Fits: Specific Models - Check Fitment Chart...,,,https://m.media-amazon.com/images/I/710nyxS7J1...,Detroit Axle,Acrylonitrile Butadiene Styrene (ABS),...,,,,,,,,,,
417,418,B079X31MPS,Detroit Axle - 4 Wheel Bearing Hubs for 2010-2...,$234.83,1. Replacement for 2010-2017 Chevrolet Equinox...,,,https://m.media-amazon.com/images/I/71p8-XJ-4Q...,Detroit Axle,Acrylonitrile Butadiene Styrene (ABS),...,,,,,,,,,,
418,419,B07B13N7LV,Detroit Axle - 2.5L Wheel Bearing Hubs for 200...,$174.69,1. Replacement for 2.5L Only\n2. Replacement f...,,,https://m.media-amazon.com/images/I/81f98CGn8T...,Detroit Axle,Acrylonitrile Butadiene Styrene (ABS),...,,,,,,,,,,
